In [ ]:
pwd

In [ ]:
import digraphs
import graphs
import csv
from digraphs import N_out, topOrdering
from digraphs import maxMatching

In [ ]:

def test_1():
    games = {('Alice', 'Bob'), ('Alice', 'Dave'),
             ('Bob', 'Charlie'), ('Dave', 'Charlie')}
    result = scores(3, 2, 4, games)
    expected = {'Alice': 8, 'Bob': 3, 'Charlie': 0, 'Dave': 3}
    assert result == expected, f'Expected {expected}, but got {result}'
    print('test_1 passed.')


def test_2():
    games = {
        ('Edward', 'Vicious'),
        ('Faye Valentine', 'Ein'),
        ('Faye Valentine', 'Vicious'),
        ('Jet Black', 'Edward'),
        ('Jet Black', 'Ein'),
        ('Jet Black', 'Vicious'),
        ('Spike Spiegel', 'Edward'),
        ('Spike Spiegel', 'Ein'),
        ('Spike Spiegel', 'Faye Valentine')
    }
    result = scores(4, 2, 8, games)
    expected = {
        'Edward': 4,
        'Ein': 0,
        'Faye Valentine': 8,
        'Jet Black': 12,
        'Spike Spiegel': 14,
        'Vicious': 0
    }
    assert result == expected, f'Expected {expected}, but got {result}'
    print('test_2 passed.')


def test_3():
    games = {
        ('Edward', 'Jet Black'),
        ('Edward', 'Spike Spiegel'),
        ('Edward', 'Vicious'),
        ('Faye Valentine', 'Ein'),
        ('Faye Valentine', 'Vicious'),
        ('Jet Black', 'Ein'),
        ('Spike Spiegel', 'Ein'),
        ('Spike Spiegel', 'Faye Valentine'),
        ('Vicious', 'Jet Black')
    }
    result = scores(4, 2, 8, games)
    expected = {
        'Edward': 16,
        'Ein': 0,
        'Faye Valentine': 10,
        'Jet Black': 4,
        'Spike Spiegel': 10,
        'Vicious': 6
    }
    assert result == expected, f'Expected {expected}, but got {result}'
    print('test_3 passed.')


def test_4():
    games = {
        ('Edward', 'Ein'),
        ('Edward', 'Spike Spiegel'),
        ('Ein', 'Vicious'),
        ('Faye Valentine', 'Edward'),
        ('Jet Black', 'Ein'),
        ('Jet Black', 'Spike Spiegel'),
        ('Spike Spiegel', 'Jet Black'),
        ('Spike Spiegel', 'Vicious'),
        ('Vicious', 'Faye Valentine')
    }
    result = scores(5, 2, 9, games)
    expected = {
        'Edward': 14,
        'Ein': 7,
        'Faye Valentine': 9,
        'Jet Black': 12,
        'Spike Spiegel': 14,
        'Vicious': 7
    }
    assert result == expected, f'Expected {expected}, but got {result}'
    print('test_4 passed.')


def test_5():
    games = {
        ('Edward', 'Ein'),
        ('Edward', 'Vicious'),
        ('Julia', 'Ein'),
        ('Ein', 'Vicious'),
        ('Faye Valentine', 'Edward'),
        ('Faye Valentine', 'Ein'),
        ('Jet Black', 'Edward'),
        ('Jet Black', 'Faye Valentine'),
        ('Jet Black', 'Julia'),
        ('Jet Black', 'Spike Spiegel'),
        ('Spike Spiegel', 'Faye Valentine'),
        ('Spike Spiegel', 'Julia'),
        ('Vicious', 'Julia'),
        ('Vicious', 'Spike Spiegel')
    }
    result = scores(5, 2, 9, games)
    expected = {
        'Edward': 14,
        'Ein': 9,
        'Faye Valentine': 12,
        'Jet Black': 24,
        'Julia': 7,
        'Spike Spiegel': 14,
        'Vicious': 14
    }
    assert result == expected, f'Expected {expected}, but got {result}'
    print('test_5 passed.')

In [ ]:
def ranking(games):
    # sets the vertices to each player
    V = set(player for match in games for player in match)

    # returns the topological ordering of the players with games being the edges
    return digraphs.topOrdering(V, games)

In [ ]:
def scores(p, s, c, games):
    # Define vertices and edges for the directed graph
    V = {player for game in games for player in game}
    E = {(winner, loser) for winner, loser in games}

    # Calculate primary wins using N_out
    primary_wins = {player: N_out(V, E, player) for player in V}

    # Calculate primary scores
    primary_scores = {player: p * len(wins)
                      for player, wins in primary_wins.items()}

    # Calculate secondary wins using NS_out and N_in
    secondary_wins = {
        player: {sec_player for opponent in primary_wins[player]
                 for sec_player in N_out(V, E, opponent) if sec_player != player}
        for player in V
    }

    # Remove primary wins from secondary wins to avoid double counting
    secondary_wins = {
        player: secondary_wins[player] - primary_wins[player] for player in V}

    # Calculate secondary scores with a cap
    secondary_scores = {
        player: min(s * len(secondary_wins[player]), c)
        for player in V
    }

    # Combine primary and secondary scores
    total_scores = {
        player: primary_scores[player] + secondary_scores[player] for player in V}

    return total_scores

In [ ]:
test_1()

In [ ]:
test_2()

In [ ]:
test_3()

In [ ]:

test_4()

In [ ]:
test_5()

In [ ]:
def scores(p, s, c, games):
    # Initialize the players' scores and wins
    players = {winner for winner, _ in games}.union(
        {loser for _, loser in games})
    primary_wins = {player: set() for player in players}
    secondary_wins = {player: set() for player in players}

    # Populate primary and secondary wins
    for winner, loser in games:
        primary_wins[winner].add(loser)
        for opponent, opp_loser in games:
            if loser == opponent:
                secondary_wins[winner].add(opp_loser)

    # Calculate scores
    scores = {player: 0 for player in players}
    for player in players:
        # Add points for primary wins
        scores[player] += p * len(primary_wins[player])

        # Add points for secondary wins
        secondary_points = 0
        for secondary in secondary_wins[player]:
            if secondary not in primary_wins[player]:
                secondary_points += s
        scores[player] += min(secondary_points, c)

    return scores

In [ ]:
def scores(p, s, c, games):
    # Define vertices and edges for the directed graph
    V = {player for game in games for player in game}
    E = {(winner, loser) for winner, loser in games}

    # Calculate primary wins using N_out
    primary_wins = {player: N_out(V, E, player) for player in V}

    # Calculate primary scores
    primary_scores = {player: p * len(wins)
                      for player, wins in primary_wins.items()}

    # Calculate secondary wins using NS_out and N_in
    secondary_wins = {
        player: {sec_player for opponent in primary_wins[player]
                 for sec_player in N_out(V, E, opponent) if sec_player != player}
        for player in V
    }

    # Remove primary wins from secondary wins to avoid double counting
    secondary_wins = {
        player: secondary_wins[player] - primary_wins[player] for player in V}

    # Calculate secondary scores with a cap
    secondary_scores = {
        player: min(s * len(secondary_wins[player]), c)
        for player in V
    }

    # Combine primary and secondary scores
    total_scores = {
        player: primary_scores[player] + secondary_scores[player] for player in V}

    return total_scores


games = {
    ('Edward', 'Ein'),
    ('Edward', 'Spike Spiegel'),
    ('Ein', 'Vicious'),
    ('Faye Valentine', 'Edward'),
    ('Jet Black', 'Ein'),
    ('Jet Black', 'Spike Spiegel'),
    ('Spike Spiegel', 'Jet Black'),
    ('Spike Spiegel', 'Vicious'),
    ('Vicious', 'Faye Valentine')
}
result = scores(5, 2, 9, games)
correct = {'Edward': 14, 'Ein': 7, 'Faye Valentine': 9,
           'Jet Black': 12, 'Spike Spiegel': 14, 'Vicious': 7}
print("correct answer: ", correct)
print("my answer: ", result)